# Задание 1 (5 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [ ]:
!mkdir data
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip -P data
!unzip -o data/corpus_wsd_50k.txt.zip -d data/

--2024-10-03 20:15:45--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_wsd_50k.txt.zip [following]
--2024-10-03 20:15:45--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_wsd_50k.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4723095 (4.5M) [application/zip]
Saving to: ‘data/corpus_wsd_50k.txt.zip’

corpus_wsd_50k.txt. 100%[===================>]   4.50M  --.-KB/s    in 0.05s   

2024-10-03 20:15:46 (

In [13]:
!python -m pip install torch torchvision torchaudio -q
!python -m pip install sentence_transformers transformers accelerate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 65.2 MB/s eta 0:00:00


In [14]:
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [15]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
corpus = open('data/corpus_wsd_50k.txt').read()
sentences = corpus.split('\n\n')
tokenized_sentences = []
for sent in sentences:
    tokenized_sent = []
    for word in sent.split('\n'):
        tokenized_sent.append(word.split('\t'))
    tokenized_sentences.append(tokenized_sent)
tokenized_sentences[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

In [ ]:
# Jaccard score
def lesk_jaccard(word, context):
    max_score = 0
    best_key = None

    for synset in wn.synsets(word):
        cur_key = synset.lemmas()[0].key()
        intersection = (set(synset.definition().split()) & set(context))
        union = (set(synset.definition().split()) | set(context))
        jaccard_score =  len(intersection) / len(union)

        if jaccard_score > max_score:
            max_score = jaccard_score
            best_key = cur_key

        return best_key

In [ ]:
true = 0
all = 0

for sent in tokenized_sentences:
    context = []
    for word in sent:
        if len(word) == 3:
            context.append(word[1])
    for word in sent:
        if word[0] != '':
            if lesk_jaccard(word[1], context) == word[0]:
                true += 1
            all += 1

In [ ]:
accuracy = true / all
accuracy

0.2521622421461113

In [ ]:
# Cosine distance
def lesk_cosine(word, context):
    min_score = 2
    best_key = None

    context_emb = embed(context)
    cur_key = []
    definition_emb = []

    for synset in wn.synsets(word):
        cur_key.append(synset.lemmas()[0].key())
        definition_emb.append(embed(synset.definition()))

    cosine_score = cosine_distances(context_emb.reshape(1, -1), definition_emb)[0]
    keys_scores = list(zip(cur_key, cosine_score))
    for key_score in keys_scores:
        if key_score[1] < min_score:
            min_score = key_score[1]
            best_key = key_score[0]

    return best_key

In [ ]:
true = 0
all = 0

for sent in tokenized_sentences[:100]:
    for s in [sent]:
        context = ''
        for word in s:
            if len(word) == 3:
                context += word[2] + ' '
        for word in s:
            if word[0] != '':
                if lesk_cosine(word[1], context) == word[0]:
                    true += 1
                all += 1

In [ ]:
accuracy = true / all
accuracy

0.34968553459119495

# Задание 2 (5 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты.

In [28]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, AgglomerativeClustering, SpectralClustering
import numpy as np
from sklearn.metrics import adjusted_rand_score

from IPython.display import Image
from IPython.core.display import HTML

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv', sep='\t')
df.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,"0-5, 339-344",замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,"11-16, 17-22, 188-193","шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,299-304,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,111-116,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,"134-139, 262-267",великий князь литовский гедимин после успешной...


In [11]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [9]:
def ARI(grouped_df, cluster):
    ARI = []
    for key, _ in grouped_df:
        texts = grouped_df.get_group(key)['context'].values
        X = np.zeros((len(texts), 768))
        for i, text in enumerate(texts):
            X[i] = embed(text)
        cluster.fit(X)
        labels = np.array(cluster.labels_)+1
        ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    print(np.mean(ARI))

In [18]:
# DBSCAN
ARI(grouped_df, DBSCAN(eps=0.3, min_samples=10))
ARI(grouped_df, DBSCAN(eps=0.5, min_samples=5))
ARI(grouped_df, DBSCAN(eps=0.8, min_samples=8))
ARI(grouped_df, DBSCAN(eps=0.2, min_samples=6))
ARI(grouped_df, DBSCAN(eps=0.6, min_samples=2))

0.0
-0.011271692824715207
-0.001784882962278153
0.0
0.13224860940742678


In [21]:
# Affinity Propagation
ARI(grouped_df, AffinityPropagation(damping=0.5, max_iter=200))
ARI(grouped_df, AffinityPropagation(damping=0.6, max_iter=400))
ARI(grouped_df, AffinityPropagation(damping=0.8, max_iter=180))
ARI(grouped_df, AffinityPropagation(damping=0.7, max_iter=260))
ARI(grouped_df, AffinityPropagation(damping=0.9, max_iter=300))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


0.04270504600809172
0.042740969848549505
0.04150923434928373
0.04154515818974152
0.05297560306165972


In [24]:
# Agglomerative Clustering
ARI(grouped_df, AgglomerativeClustering(n_clusters=2))
ARI(grouped_df, AgglomerativeClustering(n_clusters=3))
ARI(grouped_df, AgglomerativeClustering(n_clusters=8))
ARI(grouped_df, AgglomerativeClustering(n_clusters=5))
ARI(grouped_df, AgglomerativeClustering(n_clusters=6))

-0.011976265536517934
0.034738071269679226
0.057641599566266404
0.03379296274962468
0.03957581346650506


In [25]:
# KMeans
ARI(grouped_df, KMeans(n_clusters=8))
ARI(grouped_df, KMeans(n_clusters=3))
ARI(grouped_df, KMeans(n_clusters=2))
ARI(grouped_df, KMeans(n_clusters=6))
ARI(grouped_df, KMeans(n_clusters=5))

0.0692090191117664
0.08906795642825913
0.028898349558754394
0.04002832283459884
0.044974976436898254


In [29]:
# SpectralClustering
ARI(grouped_df, SpectralClustering(n_clusters=8))
ARI(grouped_df, SpectralClustering(n_clusters=3))
ARI(grouped_df, SpectralClustering(n_clusters=2))
ARI(grouped_df, SpectralClustering(n_clusters=6))
ARI(grouped_df, SpectralClustering(n_clusters=5))

0.06900188226985779
0.05690983787904186
0.0321952384748317
0.05964668221176761
0.0724454214114197


В Spectral clustering данные представляются в виде графа, где вершины соответствуют объектам, а рёбра — сходству между объектами. Затем на основе этого графа строится матрица смежности, которая описывает связи между вершинами. Эта матрица преобразуется в матрицу Лапласа, которая имеет собственные значения и собственные векторы. Эти собственные значения используются для определения количества кластеров и их центров.